In [1]:
import pandas as pd
import numpy as np
import sys
!{sys.executable} -m pip install statsmodels
import statsmodels.api as sm

You are using pip version 19.0.3, however version 20.3.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
from patsy import dmatrices
import matplotlib.pyplot as plt

In [3]:
# load latest datasets
all = pd.read_csv("https://projects.fivethirtyeight.com/soccer-api/club/spi_matches.csv")
spi = pd.read_csv("https://projects.fivethirtyeight.com/soccer-api/club/spi_global_rankings.csv")

In [102]:
all = all[all['league'] == 'German Bundesliga']
spi = spi[spi['league'] == 'German Bundesliga']

In [103]:
# current match
newgame = {'score1': 0,
           'score2': 0,
           'team1': 'Bayern Munich',
           'team2': 'Mainz',
           'spi1': float(spi.spi[spi.name == 'Bayern Munich']),
           'spi2': float(spi.spi[spi.name == 'Mainz'])
        }

In [104]:
all = all.append(newgame, ignore_index=True)

In [105]:
#n = len(all)
#mask = np.array([True]*n)
#mask[len(all)-1] = False

mask = np.random.rand(len(all)) < 0.8
mask[len(all)-1] = False
df_train = all[mask]
df_test = all[~mask]
print('Training data set length='+str(len(df_train)))
print('Testing data set length='+str(len(df_test)))

Training data set length=1218
Testing data set length=313


In [106]:
expr1 = """score1 ~ team1  + team2 + spi1 + spi2"""
expr2 = """score2 ~ team1  + team2 + spi1 + spi2"""

In [108]:
y_train1, X_train1 = dmatrices(expr1, df_train, return_type='dataframe')
y_test1, X_test1 = dmatrices(expr1, df_test, return_type='dataframe')
y_train2, X_train2 = dmatrices(expr2, df_train, return_type='dataframe')
y_test2, X_test2 = dmatrices(expr2, df_test, return_type='dataframe')

In [109]:
poisson_training_results1 = sm.GLM(y_train1, X_train1, family=sm.families.Poisson()).fit()
poisson_training_results2 = sm.GLM(y_train2, X_train2, family=sm.families.Poisson()).fit()

In [110]:
poisson_predictions1 = poisson_training_results1.get_prediction(X_test1)
#.summary_frame() returns a pandas DataFrame
predictions_summary_frame1 = poisson_predictions1.summary_frame()
print(predictions_summary_frame1)

poisson_predictions2 = poisson_training_results2.get_prediction(X_test2)
#.summary_frame() returns a pandas DataFrame
predictions_summary_frame2 = poisson_predictions2.summary_frame()
print(predictions_summary_frame2)

          mean   mean_se  mean_ci_lower  mean_ci_upper
3     2.061110  0.446159       1.348483       3.150335
14    1.225542  0.302919       0.754981       1.989393
20    0.772612  0.191288       0.475571       1.255184
21    1.575233  0.241697       1.166112       2.127891
24    1.976612  0.275011       1.504845       2.596277
...        ...       ...            ...            ...
1330  1.044820  0.484219       0.421264       2.591364
1332  1.609803  0.326468       1.081806       2.395501
1339  1.212258  0.185727       0.897810       1.636838
1340  1.692224  0.316063       1.173480       2.440282
1530  3.994039  0.480172       3.155578       5.055286

[278 rows x 4 columns]
          mean   mean_se  mean_ci_lower  mean_ci_upper
3     0.885501  0.303534       0.452285       1.733666
14    1.587715  0.384291       0.987978       2.551513
20    1.254793  0.291754       0.795532       1.979185
21    1.511266  0.249078       1.094084       2.087521
24    1.414633  0.225770       1.034654  